In [58]:
!pip install torch torchvision

In [59]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import vgg16
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

In [60]:
# Load pre-trained VGG16 model and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16_model = vgg16(pretrained=True).to(device)
vgg16_model = nn.Sequential(*list(vgg16_model.features.children()))

# Set the model to evaluation mode
vgg16_model.eval()

/home/luto_pc/Documents/btp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luto_pc/Documents/btp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [61]:
def extract_vgg_features(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = torch.unsqueeze(image, 0).to(device)  # Add batch dimension and move to GPU

    # Forward pass to get features
    with torch.no_grad():
        features = vgg16_model(image)

    # Return the features
    return features

In [62]:
image_path = './images/Test_Image_1_5.jpg'
features = extract_vgg_features(image_path)
print("VGG16 Features Shape:", features.shape)

VGG16 Features Shape: torch.Size([1, 512, 7, 7])


In [63]:
images = os.listdir('./images')
images = sorted(images,key= lambda x: int(x.split('_')[2])*100000 + int(x.split('_')[3][:-4]))

In [66]:
images[0]
features = []

In [67]:
for image in tqdm(images):
    image_path = os.path.join('./images', image)
    feature = extract_vgg_features(image_path)
    features.append(feature)

100%|██████████| 1124/1124 [00:27<00:00, 40.14it/s]


In [ ]:
numpy_array = np.stack([tensor.cpu().numpy() for tensor in features])

In [ ]:
import h5py

In [ ]:
output_file = 'features.h5'
with h5py.File(output_file, 'w') as hf:
    hf.create_dataset('features', data=numpy_array)